In [5]:
import cv2
from cnn import CNN
import tensorflow as tf
cnn = CNN()

In [6]:
data_list = [
    {"folder":"img_face","label":"0"},
    {"folder":"img_face_non_obama","label":"1"}
]
    
    
test_list = [
    {"folder":"img_face_test", "label":"0"},
    {"folder":"img_face_test_non_obama", "label":"1"}
]

train_image, train_label, _ = cnn.read_image_cv(data_list)
test_image , test_label , test_image_list = cnn.read_image_cv(test_list)  

In [8]:
with tf.Graph().as_default():
        
    images_placeholder = tf.placeholder("float", shape=(None, cnn.IMAGE_PIXELS))
    labels_placeholder = tf.placeholder("float", shape=(None, cnn.NUM_CLASSES))
    keep_prob = tf.placeholder("float")

    
    softmax = cnn.inference(images_placeholder, keep_prob)
    loss_value = cnn.loss(softmax, labels_placeholder)
    train_op = cnn.training(loss_value, cnn.LEARNING_RATE)
    acc = cnn.accuracy(softmax, labels_placeholder)
    
    saver = tf.train.Saver()

    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
            
    summary_op = tf.summary.merge_all()
    summary_writer = tf.summary.FileWriter(cnn.LOG_DIR, sess.graph)
        
        
    for step in range(cnn.MAX_STEPS):
        for i in range(len(train_image)/cnn.BATCH_SIZE):
            batch = cnn.BATCH_SIZE*i
            sess.run(train_op, feed_dict={
                    images_placeholder: train_image[batch:batch+cnn.BATCH_SIZE],
                    labels_placeholder: train_label[batch:batch+cnn.BATCH_SIZE],
                    keep_prob: 0.5})

        #train_accuracy = sess.run(acc, feed_dict={
        #        images_placeholder: train_image,
        #        labels_placeholder: train_label,
        #        keep_prob: 1.0})
        #print "step %d, training accuracy %g"%(step, train_accuracy)
               
        train_loss = sess.run(loss_value,  feed_dict={
            images_placeholder: train_image,
            labels_placeholder: train_label,
            keep_prob: 1.0})
        print "step %d, xentropy %s"%(step, str(train_loss))
            
        if step % 10 == 0 and step > 0 and train_loss < 5.0:
            cnn.display_test_prob(sess, softmax, test_image, "obama", images_placeholder,keep_prob)

        summary_str = sess.run(summary_op, feed_dict={
            images_placeholder: train_image,
            labels_placeholder: train_label,
            keep_prob: 1.0})
        summary_writer.add_summary(summary_str, step)

  
    print "test accuracy %g"%sess.run(acc, feed_dict={
        images_placeholder: test_image,
        labels_placeholder: test_label,
        keep_prob: 1.0})
    

    save_path = saver.save(sess, "model.ckpt")

step 0, xentropy 1956.51
step 1, xentropy 137.334
step 2, xentropy 291.723
step 3, xentropy 167.502
step 4, xentropy 96.1564
step 5, xentropy 117.463
step 6, xentropy 86.47
step 7, xentropy 81.92
step 8, xentropy 68.7266
step 9, xentropy 57.3259
step 10, xentropy 49.4988
step 11, xentropy 42.4368
step 12, xentropy 33.3557
step 13, xentropy 35.6241
step 14, xentropy 27.7228


KeyboardInterrupt: 

In [10]:
import getface
image_path = "/home/tamotoyoshifumi/Downloads" 
image_name = "cat.jpg"
dist_path = "/home/tamotoyoshifumi/Downloads"
getface.cutout_face(image_path, image_name, dist_path)

{'code': 1, 'http_status': 400, 'status': 'error'}